In [1]:
!pip install zyndai-agent paho-mqtt python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 107.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.5/587.5 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.3/340.3 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.8/175.8 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.5/102.5 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 79.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 k

In [2]:
import asyncio
import os
import time

In [3]:
class MockZyndNetwork:
    def __init__(self):
        self.subscribers = {}

    async def publish(self, topic, message):
        print(f"\n📡 PUBLISH → {topic}")
        for callback in self.subscribers.get(topic, []):
            await callback(topic, message)

    async def listen(self, topic, callback):
        self.subscribers.setdefault(topic, []).append(callback)

network = MockZyndNetwork()

In [4]:
class BaseAgent:
    def __init__(self, did, role, credentials):
        self.did = did
        self.role = role
        self.credentials = credentials

    def verify_agent(self, agent):
        return "zynd_verified" in agent.credentials

    def log(self, msg):
        print(f"[{self.role.upper()} | {self.did}] {msg}")

In [5]:
class AmbulanceAgent(BaseAgent):
    async def start(self):
        self.log("Connected to Zynd Network")
        await network.listen("hospital.response", self.handle_hospital_response)

        emergency_data = {
            "severity": "CRITICAL",
            "location": "NH-48 Highway",
            "vitals": {"bp": "90/60", "pulse": 120}
        }

        self.log("Broadcasting emergency request")
        await network.publish("ambulance.request", emergency_data)

    async def handle_hospital_response(self, topic, message):
        self.log(f"Hospital confirmed: {message}")

In [6]:
class HospitalAgent(BaseAgent):
    def __init__(self, did, role, credentials, available_beds):
        super().__init__(did, role, credentials)
        self.available_beds = available_beds

    async def start(self):
        self.log("Online & awaiting emergencies")
        await network.listen("ambulance.request", self.handle_ambulance_request)

    async def handle_ambulance_request(self, topic, message):
        self.log("Received emergency request")

        response = {
            "hospital": self.did,
            "available_beds": self.available_beds,
            "status": "ACCEPTED"
        }

        await network.publish("hospital.response", response)

        await network.publish("doctor.notify", {
            "patient_data": message,
            "hospital": self.did
        })

In [7]:
class DoctorAgent(BaseAgent):
    async def start(self):
        self.log("Standing by")
        await network.listen("doctor.notify", self.handle_notification)

    async def handle_notification(self, topic, message):
        self.log("Patient data received")
        self.log("Preparing treatment plan")

In [8]:
ambulance = AmbulanceAgent(
    did="did:zynd:ambulance:001",
    role="ambulance",
    credentials=["zynd_verified", "emergency_responder"]
)

hospital = HospitalAgent(
    did="did:zynd:hospital:001",
    role="hospital",
    credentials=["zynd_verified", "medical_facility"],
    available_beds=3
)

doctor = DoctorAgent(
    did="did:zynd:doctor:001",
    role="doctor",
    credentials=["zynd_verified", "trauma_specialist"]
)

In [9]:
async def run_demo():
    await asyncio.gather(
        ambulance.start(),
        hospital.start(),
        doctor.start()
    )

await run_demo()

[AMBULANCE | did:zynd:ambulance:001] Connected to Zynd Network
[AMBULANCE | did:zynd:ambulance:001] Broadcasting emergency request

📡 PUBLISH → ambulance.request
[HOSPITAL | did:zynd:hospital:001] Online & awaiting emergencies
[DOCTOR | did:zynd:doctor:001] Standing by
